<a href="https://colab.research.google.com/github/talkin24/jaxflax_lab/blob/main/Training_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch normalization

배치 정규화는 훈련 속도를 높이고 수렴을 개선하는 데 사용되는 정규화 기법. 훈련 중에 특징 차원에 대한 러닝에버리지를 계산합니다. 이렇게 하면 적절하게 처리해야 하는 새로운 형태의 미분불가능 상태가 추가됩니다.

## Defining the model with `BatchNorm`

Flax에서 `BatchNorm`은 훈련과 추론 간에 서로 다른 런타임 동작을 보이는 `flax.linen.Module`. 아래 그림과 같이 use_running_average 인수를 통해 명시적으로 지정 가능

일반적인 패턴은 부모 Flax 모듈에서 train(훈련) 인수를 수락하고, 이를 사용하여 BatchNorm의 use_running_average 인수를 정의할 수 있음

참고: PyTorch나 TensorFlow(Keras)와 같은 다른 머신 러닝 프레임워크에서는 변경 가능한 상태 또는 호출 플래그를 통해 지정합니다(예: torch.nn.Module.eval 또는 tf.keras.Model에서 훈련 플래그를 설정).


In [ ]:
# No BatchNorm
class MLP(nn.Module):
  @nn.compact
  def __call__(self, x):
    x = nn.Dense(features=4)(x)

    x = nn.relu(x)
    x = nn.Dense(features=1)(x)
    return x

In [ ]:
# With BatchNorm
class MLP(nn.Module):
  @nn.compact
  def __call__(self, x, train: bool):
    x = nn.Dense(features=4)(x)
    x = nn.BatchNorm(use_running_average=not train)(x)
    x = nn.relu(x)
    x = nn.Dense(features=1)(x)
    return x

모델을 생성한 후에는 `flax.linen.init()`을 호출하여 변수 구조를 가져와 초기화함. 여기서 BatchNorm이 없는 코드와 BatchNorm이 있는 코드의 주요 차이점은 train 인수를 제공해야 한다는 것.

## The `batch_stats` collection

`params` 컬렉션 외에도, BatchNorm은 배치 통계의 러닝 에버리지를 포함하는 `batch_stats` 컬렉션도 추가

참고: 자세한 내용은 `flax.linen` 변수 API 문서에서 확인할 수 있음

나중에 사용할 수 있도록 변수에서 `batch_stats` 컬렉션을 추출해야 함

In [ ]:
# No BatchNorm
mlp = MLP()
x = jnp.ones((1, 3))
variables = mlp.init(jax.random.PRNGKey(0), x)
params = variables['params']


jax.tree_util.tree_map(jnp.shape, variables)

In [ ]:
# Wtih BatchNorm
mlp = MLP()
x = jnp.ones((1, 3))
variables = mlp.init(jax.random.PRNGKey(0), x, train=False)
params = variables['params']
batch_stats = variables['batch_stats']

jax.tree_util.tree_map(jnp.shape, variables)

Flax `BatchNorm`은 총 4개의 변수를 추가합니다. `batch_stats` 컬렉션에 있는 `mean`과 `var`, 그리고 `params` 컬렉션에 있는 `scale`와 `bias`입니다.

In [ ]:
# No BatchNorm
FrozenDict({






  'params': {




    'Dense_0': {
        'bias': (4,),
        'kernel': (3, 4),
    },
    'Dense_1': {
        'bias': (1,),
        'kernel': (4, 1),
    },
  },
})

In [ ]:
# with BatchNorm
FrozenDict({
  'batch_stats': {
    'BatchNorm_0': {
        'mean': (4,),
        'var': (4,),
    },
  },
  'params': {
    'BatchNorm_0': {
        'bias': (4,),
        'scale': (4,),
    },
    'Dense_0': {
        'bias': (4,),
        'kernel': (3, 4),
    },
    'Dense_1': {
        'bias': (1,),
        'kernel': (4, 1),
    },
  },
})

## Modifying `flax.linen.apply`

`flax.linen.apply`를 사용하여 `train==True` 인수를 사용하여 모델을 실행할 때(즉, `BatchNorm` 호출에 `use_running_average==False`가 있는 경우) 다음 사항을 고려해야 합니다:

- `batch_stats`를 입력 변수로 전달해야 합니다.

- `batch_stats` 컬렉션은 `mutable=['batch_stats']`를 설정하여 변경 가능한 것으로 표시해야 합니다.

- 변경된 변수는 두 번째 출력으로 반환됩니다 . 업데이트된 `batch_stats`는 여기에서 추출해야 합니다.

In [ ]:
# No BatchNorm
y = mlp.apply(
  {'params': params},
  x,

)
...

In [ ]:
# With BatchNorm
y, updates = mlp.apply(
  {'params': params, 'batch_stats': batch_stats},
  x,
  train=True, mutable=['batch_stats']
)
batch_stats = updates['batch_stats']

## Training and evaluation

`BatchNorm`을 사용하는 모델을 훈련 루프에 통합할 때 가장 어려운 점은 추가적인 `batch_stats` 상태를 처리하는 것입니다. 이렇게 하려면 다음을 수행해야 합니다:

- 사용자 정의 `flax.training.train_state.TrainState` 클래스에 batch_stats 필드를 추가합니다.

- `batch_stats` 값을 `train_state.TrainState.create` 메서드에 전달합니다.

In [ ]:
# No BatchNorm
from flax.training import train_state




state = train_state.TrainState.create(
  apply_fn=mlp.apply,
  params=params,

  tx=optax.adam(1e-3),
)

In [ ]:
# With BatchNorm
from flax.training import train_state

class TrainState(train_state.TrainState):
  batch_stats: Any

state = TrainState.create(
  apply_fn=mlp.apply,
  params=params,
  batch_stats=batch_stats,
  tx=optax.adam(1e-3),
)

또한 이러한 변경 사항을 반영하도록 `train_step` 함수를 업데이트하세요:

- 이전에 설명한 대로 모든 새 매개변수를 `flax.linen.apply`에 전달합니다.

- `batch_stats`에 대한 `updates`는 `loss_fn`에서 전파되어야 합니다.

- `TrainState`의 `batch_stats`를 업데이트해야 합니다.

In [ ]:
# No BatchNorm
@jax.jit
def train_step(state: TrainState, batch):
  """Train for a single step."""
  def loss_fn(params):
    logits = state.apply_fn(
      {'params': params},
      x=batch['image'])
    loss = optax.softmax_cross_entropy_with_integer_labels(
      logits=logits, labels=batch['label'])
    return loss, logits
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)

  metrics = {
    'loss': loss,
      'accuracy': jnp.mean(jnp.argmax(logits, -1) == batch['label']),
  }
  return state, metrics

In [ ]:
# With BatchNorm
@jax.jit
def train_step(state: TrainState, batch):
  """Train for a single step."""
  def loss_fn(params):
    logits, updates = state.apply_fn(
      {'params': params, 'batch_stats': state.batch_stats},
      x=batch['image'], train=True, mutable=['batch_stats'])
    loss = optax.softmax_cross_entropy_with_integer_labels(
      logits=logits, labels=batch['label'])
    return loss, (logits, updates)
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, (logits, updates)), grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  state = state.replace(batch_stats=updates['batch_stats'])
  metrics = {
    'loss': loss,
      'accuracy': jnp.mean(jnp.argmax(logits, -1) == batch['label']),
  }
  return state, metrics

`eval_step`은 훨씬 간단합니다. `batch_stats`는 변경 가능하지 않으므로 업데이트를 전파할 필요가 없습니다. `batch_stats`를 `flax.linen.apply`에 전달하고 `train` 인수가 `False`로 설정되어 있는지 확인하세요

In [ ]:
@jax.jit
def eval_step(state: TrainState, batch):
  """Train for a single step."""
  logits = state.apply_fn(
    {'params': params},
    x=batch['image'])
  loss = optax.softmax_cross_entropy_with_integer_labels(
    logits=logits, labels=batch['label'])
  metrics = {
    'loss': loss,
      'accuracy': jnp.mean(jnp.argmax(logits, -1) == batch['label']),
  }
  return state, metrics

In [ ]:
@jax.jit
def eval_step(state: TrainState, batch):
  """Train for a single step."""
  logits = state.apply_fn(
    {'params': params, 'batch_stats': state.batch_stats},
    x=batch['image'], train=False)
  loss = optax.softmax_cross_entropy_with_integer_labels(
    logits=logits, labels=batch['label'])
  metrics = {
    'loss': loss,
      'accuracy': jnp.mean(jnp.argmax(logits, -1) == batch['label']),
  }
  return state, metrics

# Dropout

이 가이드에서는 `flax.linen.Dropout()`을 사용하여 드롭아웃을 적용하는 방법에 대한 개요를 제공합니다.

드롭아웃은 네트워크에서 숨겨진 단위와 보이는 단위를 무작위로 제거하는 확률적 정규화 기법입니다.

가이드 전체에서 Flax `Dropout`을 적용한 코드 예제와 적용하지 않은 코드 예제를 비교할 수 있습니다.

## Split the PRNG key

드롭아웃은 무작위 연산이므로 의사 난수 생성기(PRNG) 상태가 필요합니다. Flax는 중립 네트워크에 바람직한 여러 가지 속성을 가진 JAX의 (분할 가능한) PRNG 키를 사용합니다. 자세한 내용은 JAX 튜토리얼의 의사 난수를 참조하세요.

참고: JAX에는 PRNG 키를 제공하는 명시적인 방법이 있다는 것을 기억하세요. `key, subkey = jax.random.split(key)`를 사용하여 기본 PRNG 상태(예: `key = jax.random.PRNGKey(seed=0)`)를 여러 개의 새 PRNG 키로 포크할 수 있습니다. 기억을 새로 고치려면 🔪 JAX - 날카로운 비트 🔪 무작위성 및 PRNG 키에서 확인할 수 있습니다.

jax.random.split()을 사용하여 PRNG 키를 Linen `Dropout`용 키를 포함하여 3개의 키로 분할하는 것으로 시작합니다.

In [ ]:
# No Dropout
root_key = jax.random.PRNGKey(seed=0)
main_key, params_key = jax.random.split(key=root_key)

In [ ]:
# With Dropout
root_key = jax.random.PRNGKey(seed=0)
main_key, params_key, dropout_key = jax.random.split(key=root_key, num=3)

참고: Flax에서는 나중에 `flax.linen.Module()`에서 사용할 수 있도록 PRNG 스트림에 이름을 제공합니다. 예를 들어, 매개변수를 초기화하기 위해 `'params'` 스트림을 전달하고 `flax.linen.Dropout()`을 적용하기 위해 `'dropout'`을 전달합니다.

## Define your model with `Dropout`


드롭아웃이 있는 모델을 만들려면:

- `flax.linen.Module()`을 서브클래싱한 다음 `flax.linen.Dropout()`을 사용하여 드롭아웃 레이어를 추가합니다. `flax.linen.Module()`은 모든 신경망 모듈의 베이스 클래스이며, 모든 레이어와 모델은 이 클래스에서 서브클래스화된다는 점을 기억하세요.

- `flax.linen.Dropout()`에서 `deterministic` 인수는 키워드 인자로도 전달해야 합니다:

  - `flax.linen.Module()`을 구성할 때; 또는
  - 생성된 모듈에서 `flax.linen.init()` 또는 `flax.linen.apply()`를 호출할 때. (자세한 내용은 `flax.linen.module.merge_param()` 참조).

-  `deterministic`은 boolean이므로:

  - `False`로 설정하면 입력이 마스킹되고(즉, 0으로 설정됨) `rate`에 따라 확률이 설정됩니다. 그리고 나머지 입력은 `1 / (1 - 비율)`로 스케일링되어 입력의 평균이 유지됩니다.

  - `True`로 설정하면 마스크가 적용되지 않고(드롭아웃이 꺼짐) 입력이 그대로 반환됩니다.

일반적인 패턴은 부모 Flax `Module`에서 `Training`(또는 `train`) 인수(boolean)를 받아 이를 사용하여 드롭아웃을 활성화 또는 비활성화하는 것입니다(이 가이드의 뒷부분에서 설명). PyTorch나 TensorFlow(Keras)와 같은 다른 머신 러닝 프레임워크에서는 변경 가능한 상태 또는 호출 플래그를 통해 지정합니다(예: `torch.nn.Module.eval` 또는 `tf.keras.Model`에서 훈련 플래그를 설정하여).

참고: Flax는 `Flax flax.linen.Module()`의 `flax.linen.Module.make_rng()` 메서드를 통해 PRNG 키 스트림을 암시적으로 처리하는 방법을 제공합니다. 이를 통해 Flax 모듈(또는 그 하위 모듈) 내부의 새로운 PRNG 키를 PRNG 스트림에서 분리할 수 있습니다. `make_rng` 메서드는 호출할 때마다 고유한 키를 제공하도록 보장합니다. 내부적으로 `flax.linen.Dropout()`은 `flax.linen.Module.make_rng()`를 사용하여 드롭아웃을 위한 키를 생성합니다. 소스 코드를 확인할 수 있습니다. 요컨대, `flax.linen.Module.make_rng()`는 완전한 재현성을 보장합니다.

In [ ]:
# No Dropout
class MyModel(nn.Module):
  num_neurons: int

  @nn.compact
  def __call__(self, x):
    x = nn.Dense(self.num_neurons)(x)



    return x

In [ ]:
# With Dropout
class MyModel(nn.Module):
  num_neurons: int

  @nn.compact
  def __call__(self, x, training: bool):
    x = nn.Dense(self.num_neurons)(x)
    # Set the dropout layer with a `rate` of 50%.
    # When the `deterministic` flag is `True`, dropout is turned off.
    x = nn.Dropout(rate=0.5, deterministic=not training)(x)
    return x

## Initialize the model

모델을 생성한 후

- 모델을 인스턴스화합니다.
- 그런 다음 `flax.linen.init()` 호출에서 `training=False`를 설정합니다.
- 마지막으로 변수 사전에서 `params`를 추출합니다.

여기서 Flax `Dropout`이 없는 코드와 `Dropout`이 있는 코드의 주요 차이점은 드롭아웃을 활성화해야 하는 경우 `training`(또는 `train`) 인수를 제공해야 한다는 것입니다.

In [ ]:
# No Dropout
my_model = MyModel(num_neurons=3)
x = jnp.empty((3, 4, 4))

variables = my_model.init(params_key, x)
params = variables['params']

In [ ]:
# With Dropout
my_model = MyModel(num_neurons=3)
x = jnp.empty((3, 4, 4))
# Dropout is disabled with `training=False` (that is, `deterministic=True`).
variables = my_model.init(params_key, x, training=False)
params = variables['params']

## Perform the forward pass during training

`flax.linen.apply()`를 사용하여 모델을 실행할 때:

- `flax.linen.apply()`에 `training=True`를 전달합니다.
- 그런 다음 포워드 패스(드롭아웃 포함) 중에 PRNG 키를 그리려면 `flax.linen.apply()`를 호출할 때 `'dropout'` 스트림을 시드할 PRNG 키를 제공하세요.

In [ ]:
# No Dropout
# No need to pass the `training` and `rngs` flags.
y = my_model.apply({'params': params}, x)

In [ ]:
# With Dropout
# Dropout is enabled with `training=True` (that is, `deterministic=False`).
y = my_model.apply({'params': params}, x, training=True, rngs={'dropout': dropout_key})

여기서 flax `Dropout`이 없는 코드와 `Dropout`이 있는 코드의 주요 차이점은 드롭아웃을 활성화해야 하는 경우 `training`(또는 `train`) 및 `rngs` 인수를 제공해야 한다는 점입니다.

평가 중에는 드롭아웃을 활성화하지 않은 상태로 위의 코드를 사용합니다(즉, RNG도 전달할 필요가 없습니다).

## `TrainState` and the training step

이 섹션에서는 드롭아웃을 활성화한 경우 학습 단계 함수 내에서 코드를 수정하는 방법을 설명합니다.

참고: Flax에는 파라미터와 옵티마이저 상태를 포함하여 전체 학습 상태를 나타내는 데이터 클래스를 생성하는 일반적인 패턴이 있다는 점을 기억하세요. 그런 다음 단일 파라미터인 `state: TrainState`를 훈련 단계 함수에 전달할 수 있습니다. 자세한 내용은 `flax.training.train_state.TrainState()` API 문서를 참조하세요.

- 먼저 사용자 정의 `flax.training.train_state.TrainState()` 클래스에 `key` 필드를 추가합니다.

- 그런 다음 키 값(이 경우 `dropout_key`)을 `train_state.TrainState.create()` 메서드에 전달합니다.

In [ ]:
# No Dropout
from flax.training import train_state




state = train_state.TrainState.create(
  apply_fn=my_model.apply,
  params=params,

  tx=optax.adam(1e-3)
)

In [ ]:
# With Dropout
from flax.training import train_state

class TrainState(train_state.TrainState):
  key: jax.random.KeyArray

state = TrainState.create(
  apply_fn=my_model.apply,
  params=params,
  key=dropout_key,
  tx=optax.adam(1e-3)
)

- 다음으로, Flax 트레이닝 단계 함수인 `train_step`에서 `dropout_key`에서 새 PRNG 키를 생성하여 각 단계에 드롭아웃을 적용합니다. 이 작업은 다음 중 하나를 사용하여 수행할 수 있습니다:

  - `jax.random.split()`; 또는
  - `jax.random.fold_in()`

  일반적으로 `jax.random.fold_in()`을 사용하는 것이 더 빠릅니다. `jax.random.split()`을 사용하면 나중에 재사용할 수 있는 PRNG 키를 분할합니다. 그러나 `jax.random.fold_in()`을 사용하면 1) 고유한 데이터를 접어야 하며, 2) PRNG 스트림의 시퀀스를 길게 할 수 있습니다.

- 마지막으로, 포워드 패스를 수행할 때 새 PRNG 키를 `state.apply_fn()`에 추가 파라미터로 전달합니다.


In [ ]:
# No Dropout
@jax.jit
def train_step(state: TrainState, batch):

  def loss_fn(params):
    logits = state.apply_fn(
      {'params': params},
      x=batch['image'],


      )
    loss = optax.softmax_cross_entropy_with_integer_labels(
      logits=logits, labels=batch['label'])
    return loss, logits
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  return state

In [ ]:
# With Dropout
@jax.jit
def train_step(state: TrainState, batch, dropout_key):
  dropout_train_key = jax.random.fold_in(key=dropout_key, data=state.step)
  def loss_fn(params):
    logits = state.apply_fn(
      {'params': params},
      x=batch['image'],
      training=True,
      rngs={'dropout': dropout_train_key}
      )
    loss = optax.softmax_cross_entropy_with_integer_labels(
      logits=logits, labels=batch['label'])
    return loss, logits
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  return state

## Flax examples with dropout

- WMT 기계 번역 데이터셋으로 학습된 트랜스포머 기반 모델입니다. 이 예에서는 드롭아웃과 주의 드롭아웃을 사용합니다.

- 텍스트 분류 컨텍스트에서 입력 ID 배치에 단어 드롭아웃을 적용합니다. 이 예에서는 사용자 지정 `flax.linen.Dropout()` 레이어를 사용합니다.

## More Flax examples that use Module `make_rng()`
- 시퀀스 간 모델의 디코더에서 예측 토큰을 정의합니다.

# Learning rate scheduling

learning rate는 심층 신경망을 훈련하는 데 가장 중요한 하이퍼파라미터 중 하나로 간주되지만, 이를 선택하기는 매우 어려울 수 있습니다. 단순히 고정된 학습 속도를 사용하는 대신 학습 속도 스케줄러를 사용하는 것이 일반적입니다. 이 예에서는 코사인 스케줄러를 사용하겠습니다. 코사인 스케줄러가 작동하기 전에 `warmup_epochs` 에포크에 대해 학습률이 선형적으로 증가하는 소위 워밍업 기간으로 시작합니다. 코사인 스케줄러에 대한 자세한 내용은 "SGDR: 웜 재시작을 사용한 확률적 경사 하강" 문서를 참조하세요.


다음을 수행하는 방법을 보여 드리겠습니다.
- 학습 속도 일정 정의하기
- 해당 스케줄을 사용하여 간단한 모델 훈련하기

In [ ]:
def create_learning_rate_fn(config, base_learning_rate, steps_per_epoch):
  """Creates learning rate schedule."""
  warmup_fn = optax.linear_schedule(
      init_value=0., end_value=base_learning_rate,
      transition_steps=config.warmup_epochs * steps_per_epoch)
  cosine_epochs = max(config.num_epochs - config.warmup_epochs, 1)
  cosine_fn = optax.cosine_decay_schedule(
      init_value=base_learning_rate,
      decay_steps=cosine_epochs * steps_per_epoch)
  schedule_fn = optax.join_schedules(
      schedules=[warmup_fn, cosine_fn],
      boundaries=[config.warmup_epochs * steps_per_epoch])
  return schedule_fn

스케줄을 사용하려면 `create_learning_rate_fn` 함수에 하이퍼파라미터를 전달하여 learning rate 함수를 생성한 다음 이 함수를 (train_state의)`Optax` 옵티마이저에 전달해야 합니다. 예를 들어 MNIST에서 이 스케줄을 사용하려면 `train_step` 함수를 변경해야 합니다:

In [ ]:
# Default learning rate
@jax.jit
def train_step(state, batch):
  def loss_fn(params):
    logits = CNN().apply({'params': params}, batch['image'])
    one_hot = jax.nn.one_hot(batch['label'], 10)
    loss = jnp.mean(optax.softmax_cross_entropy(logits, one_hot))
    return loss, logits
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (_, logits), grads = grad_fn(state.params)
  new_state = state.apply_gradients(grads=grads)
  metrics = compute_metrics(logits, batch['label'])


  return new_state, metrics

In [ ]:
# Learning rate schedule
@functools.partial(jax.jit, static_argnums=2)
def train_step(state, batch, learning_rate_fn):
  def loss_fn(params):
    logits = CNN().apply({'params': params}, batch['image'])
    one_hot = jax.nn.one_hot(batch['label'], 10)
    loss = jnp.mean(optax.softmax_cross_entropy(logits, one_hot))
    return loss, logits
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (_, logits), grads = grad_fn(state.params)
  new_state = state.apply_gradients(grads=grads)
  metrics = compute_metrics(logits, batch['label'])
  lr = learning_rate_fn(state.step)
  metrics['learning_rate'] = lr
  return new_state, metrics

그리고 train_epoch 함수:

In [ ]:
# Default learning rate
def train_epoch(state, train_ds, batch_size, epoch, rng):
  """Trains for a single epoch."""
  train_ds_size = len(train_ds['image'])
  steps_per_epoch = train_ds_size // batch_size
  perms = jax.random.permutation(rng, len(train_ds['image']))
  perms = perms[:steps_per_epoch * batch_size]
  perms = perms.reshape((steps_per_epoch, batch_size))
  batch_metrics = []
  for perm in perms:
    batch = {k: v[perm, ...] for k, v in train_ds.items()}
    state, metrics = train_step(state, batch)
    batch_metrics.append(metrics)

  # compute mean of metrics across each batch in epoch.
  batch_metrics = jax.device_get(batch_metrics)
  epoch_metrics = {
      k: np.mean([metrics[k] for metrics in batch_metrics])
      for k in batch_metrics[0]}

  logging.info('train epoch: %d, loss: %.4f, accuracy: %.2f', epoch,
               epoch_metrics['loss'], epoch_metrics['accuracy'] * 100)

  return state, epoch_metrics

In [ ]:
# Learning rate schedule
def train_epoch(state, train_ds, batch_size, epoch, learning_rate_fn, rng):
  """Trains for a single epoch."""
  train_ds_size = len(train_ds['image'])
  steps_per_epoch = train_ds_size // batch_size
  perms = jax.random.permutation(rng, len(train_ds['image']))
  perms = perms[:steps_per_epoch * batch_size]
  perms = perms.reshape((steps_per_epoch, batch_size))
  batch_metrics = []
  for perm in perms:
    batch = {k: v[perm, ...] for k, v in train_ds.items()}
    state, metrics = train_step(state, batch, learning_rate_fn)
    batch_metrics.append(metrics)

  # compute mean of metrics across each batch in epoch.
  batch_metrics = jax.device_get(batch_metrics)
  epoch_metrics = {
      k: np.mean([metrics[k] for metrics in batch_metrics])
      for k in batch_metrics[0]}

  logging.info('train epoch: %d, loss: %.4f, accuracy: %.2f', epoch,
               epoch_metrics['loss'], epoch_metrics['accuracy'] * 100)

  return state, epoch_metrics

그리고 `create_train_state` 함수

In [ ]:
# Default learning rate
def create_train_state(rng, config):
  """Creates initial `TrainState`."""
  cnn = CNN()
  params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params']
  tx = optax.sgd(config.learning_rate, config.momentum)
  return train_state.TrainState.create(
      apply_fn=cnn.apply, params=params, tx=tx)

In [ ]:
# Learning rate schedule
def create_train_state(rng, config, learning_rate_fn):
  """Creates initial `TrainState`."""
  cnn = CNN()
  params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params']
  tx = optax.sgd(learning_rate_fn, config.momentum)
  return train_state.TrainState.create(
      apply_fn=cnn.apply, params=params, tx=tx)

# Transfer learning


이 가이드는 Flax를 사용한 전이 학습 워크플로우의 다양한 부분을 보여줍니다. 작업에 따라 사전 학습된 모델을 feature extractor로만 사용하거나 더 큰 모델의 일부로 fine-tuned할 수 있습니다.

이 가이드에서는 그 방법을 설명합니다:

- 허깅페이스 트랜스포머에서 사전 학습된 모델을 로드하고 해당 사전 학습된 모델에서 특정 하위 모듈을 추출하는 방법을 설명합니다.
- classifier 모델을 생성합니다.
- 사전 학습된 파라미터를 새 모델 구조로 전송합니다.
- Optax를 사용하여 모델의 다른 부분을 개별적으로 훈련하기 위한 옵티마이저를 생성합니다.
- 훈련할 모델을 설정합니다.

### 성능 참고 사항
작업에 따라 이 가이드의 일부 내용이 최적이 아닐 수도 있습니다. 예를 들어, 사전 학습된 모델 위에 선형 분류기만 학습하려는 경우 특징 임베딩을 한 번만 추출하는 것이 훨씬 더 빠른 학습이 될 수 있으며 선형 회귀 또는 로지스틱 분류를 위한 특수 알고리즘을 사용할 수 있습니다. 이 가이드에서는 모든 모델 파라미터를 사용하여 전이 학습을 수행하는 방법을 보여줍니다.

## Setup

In [1]:
# Note that the Transformers library doesn't use the latest Flax version.
! pip install -q transformers[flax]
# Install/upgrade Flax and JAX. For JAX installation with GPU/TPU support,
# visit https://github.com/google/jax#installation.
! pip install -U -q flax jax jaxlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.8/936.8 kB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.2/214.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.6/185.6 kB 12.8 MB/s eta 

## Create a function for model loading

사전 학습된 분류기를 로드하려면 편의를 위해 먼저 Flax `Module`과 사전 학습된 변수를 반환하는 함수를 만듭니다.

아래 코드에서 `load_model` 함수는 트랜스포머 라이브러리에서 HuggingFace의 `FlaxCLIPVisionModel` 모델을 사용하고 `FlaxCLIPModule` 모듈을 추출합니다.

In [2]:
%%capture
from IPython.display import clear_output
from transformers import FlaxCLIPModel

# Note: FlaxCLIPModel is not a Flax Module
def load_model():
  clip = FlaxCLIPModel.from_pretrained('openai/clip-vit-base-patch32')
  clear_output(wait=False) # Clear the loading messages
  module = clip.module # Extract the Flax Module
  variables = {'params': clip.params} # Extract the parameters
  return module, variables

`FlaxCLIPVisionModel` 자체는 Flax `Module`이 아니기 때문에 이 추가 단계를 수행해야 한다는 점에 유의하세요.

## Extracting a submodule

위의 스니펫에서 `load_model`을 호출하면 `text_model` 및 `vision_model` 하위 모듈로 구성된 FlaxCLIPModule이 반환됩니다.

`.setup()` 내부에 정의된 비전 모델 하위 모듈과 그 변수를 추출하는 쉬운 방법은 `clip` 모듈에 `flax.linen.Module.bind`를 바로 뒤에 사용하고 비전 모델 하위 모듈에 `flax.linen.Module.unbind`를 사용하는 것입니다.


In [8]:
import flax.linen as nn

clip, clip_variables = load_model()
vision_model, vision_model_vars = clip.bind(clip_variables).vision_model.unbind()

## Creating a classifier

분류기를 만들려면 `backbone`(사전 학습된 비전 모델)과 `head`(분류기) 하위 모듈로 구성된 새 Flax `Module`을 정의합니다.

In [9]:
from typing import Callable
import jax.numpy as jnp
import jax

class Classifier(nn.Module):
  num_classes: int
  backbone: nn.Module
  

  @nn.compact
  def __call__(self, x):
    x = self.backbone(x).pooler_output
    x = nn.Dense(
      self.num_classes, name='head', kernel_init=nn.zeros)(x)
    return x

분류기 `model`을 구성하기 위해 비전 모델 모듈이 `Classifier`에 `backbone`으로 전달됩니다. 그런 다음 매개변수 모양을 추론하는 데 사용되는 가짜 데이터를 전달하여 모델의 `params`를 임의로 초기화할 수 있습니다.

In [10]:
num_classes = 3
model = Classifier(num_classes=num_classes, backbone=vision_model)

x = jnp.empty((1, 224, 224, 3))
variables = model.init(jax.random.PRNGKey(1), x)
params = variables['params']

## Transfer the parameters

현재 `params`는 무작위이므로 `vision_model_vars`에서 사전 학습된 파라미터를 적절한 위치의 `params` 구조로 전송해야 합니다. 이 작업은 `params` 고정을 해제하고 `backbone` 파라미터를 업데이트한 다음 `params`를 다시 고정하는 방식으로 수행할 수 있습니다:

In [11]:
from flax.core.frozen_dict import freeze

params = params.unfreeze()
params['backbone'] = vision_model_vars['params']
params = freeze(params)

참고: 모델에 `batch_stats`와 같은 다른 변수 컬렉션이 포함되어 있는 경우 이러한 컬렉션도 전송해야 합니다.

## Optimization

모델의 다른 부분을 개별적으로 훈련해야 하는 경우 세 가지 옵션이 있습니다:

1. `stop_gradient`를 사용합니다.
2. `jax.grad`에 대한 매개변수를 필터링합니다.
3. 여러 매개변수에 대해 여러 옵티마이저를 사용합니다.

대부분의 경우 효율적이고 다양한 미세 조정 전략을 구현하기 위해 쉽게 확장할 수 있으므로 Optax의 `multi_transform`을 통해 여러 옵티마이저를 사용하는 것이 좋습니다.

### Optax.multi_transform


`optax.multi_transform`을 사용하려면 다음을 정의해야 합니다:

1. 매개변수 파티션.
2. 파티션과 해당 옵티마이저 간의 매핑.
3. 매개변수와 모양은 같지만 해당 파티션 레이블이 포함된 leaves를 가진 파이트리.

위의 모델에 대해 `optax.multi_transform`을 사용하여 레이어를 고정하려면 다음 설정을 사용할 수 있습니다:

- `trainable` 파라미터와 `frozen` 파라미터 파티션을 정의합니다.
- `trainable` 파라미터의 경우 Adam(`optax.adam`) 옵티마이저를 선택합니다.
- `frozen` 파라미터의 경우 `optax.set_to_zero` 옵티마이저를 선택합니다. 이 더미 옵티마이저는 그라디언트를 0으로 설정하므로 학습이 수행되지 않습니다.
- `flax.traverse_util.path_aware_map`을 사용하여 파라미터를 파티션에 매핑하고 `backbone`의 잎은 `frozen`으로, 나머지는 `trainable`으로 표시합니다.

In [12]:
from flax import traverse_util
import optax

partition_optimizers = {'trainable': optax.adam(5e-3), 'frozen': optax.set_to_zero()}
param_partitions = freeze(traverse_util.path_aware_map(
  lambda path, v: 'frozen' if 'backbone' in path else 'trainable', params))
tx = optax.multi_transform(partition_optimizers, param_partitions)

# visualize a subset of the param_partitions structure
flat = list(traverse_util.flatten_dict(param_partitions).items())
freeze(traverse_util.unflatten_dict(dict(flat[:2] + flat[-2:])))

FrozenDict({
    backbone: {
        embeddings: {
            class_embedding: 'frozen',
            patch_embedding: {
                kernel: 'frozen',
            },
        },
    },
    head: {
        bias: 'trainable',
        kernel: 'trainable',
    },
})

`differential learning rates`를 구현하기 위해 `optax.set_to_zero`를 다른 옵티마이저로 대체할 수 있으며, 작업에 따라 다른 옵티마이저와 파티셔닝 체계를 선택할 수 있습니다. 고급 옵티마이저에 대한 자세한 내용은 옵택스의 옵티마이저 결합 문서를 참조하세요.

## Creating the `TrainState`

모듈, 파라미터, 옵티마이저가 정의되면 평소와 같이 `TrainState`를 구성할 수 있습니다:

In [13]:
from flax.training.train_state import TrainState

state = TrainState.create(
  apply_fn=model.apply,
  params=params,
  tx=tx)

optimizer가 전략의 동결 또는 미세 조정을 처리하므로 `train_step`을 추가로 변경할 필요가 없으므로 훈련을 정상적으로 진행할 수 있습니다.

# Save and load checkpoints

In [ ]:
b